 Import Libraries and Configuration

In [1]:
import serial
import csv
import time


Serial Port Configuration and File Creation

In [3]:
# Configure the serial port - adjust the port name as needed
ser = serial.Serial('COM5', 115200, timeout=1)

# Generate a unique filename based on the current timestamp
timestamp = time.strftime("%Y%m%d%H%M%S")
csv_filename = f"brainwave_data_{timestamp}.csv"
csv_file = open(csv_filename, mode='w', newline='')
csv_writer = csv.writer(csv_file)

Initialize Data Structures and Settings

In [4]:
# Initialize a list to store frequencies
frequencies = []

# Define the recording duration in seconds
recording_duration = 60  # Adjust as needed

# Define the number of samples
num_samples = 128

# Calculate the number of rows to record
num_rows_to_record = recording_duration * num_samples


Read and Process Serial Data

In [6]:
# Read and process data from the serial port
def read_serial_data():
    psd_values = [[] for _ in range(num_samples)]  # Initialize a list of empty lists for PSD values
    recorded_rows = 0
    while recorded_rows < num_rows_to_record:
        line = ser.readline().decode().strip()  # Read a line from the serial port
        values = line.split(",")
        if len(values) == 2:
            try:
                frequency = float(values[0])
                psd = float(values[1])
                frequencies.append(frequency)
                for i in range(num_samples):
                    psd_values[i].append(psd)
                recorded_rows += 1
                if recorded_rows >= num_rows_to_record:
                    save_to_csv(frequencies, psd_values)
                    print("Recording complete. Press Enter to start a new recording or Ctrl+C to exit.")
                    input()  # Wait for user input to start a new recording
            except ValueError:
                pass


Save Data to CSV

In [7]:
# Save data to the CSV file
def save_to_csv(frequencies, psd_values):
    # Write the header row with frequencies
    if not csv_writer:
        csv_writer.writerow(['Frequency (Hz)'] + frequencies)
    # Write rows with PSD values for each frequency
    for i in range(num_samples):
        csv_writer.writerow(['PSD' + str(i + 1)] + psd_values[i])
    csv_file.flush()  # Ensure data is written immediately
    frequencies.clear()
    for i in range(num_samples):
        psd_values[i].clear()


Start Data Collection

In [8]:
# Start reading and saving data
try:
    read_serial_data()
except KeyboardInterrupt:
    print("Data collection stopped.")
finally:
    # Close the serial port and CSV file when done
    ser.close()
    csv_file.close()


Data collection stopped.
